In [1]:
import csv
import pandas as pd
import numpy as np


Extracting dataset for training

In [2]:
df=pd.read_csv('IMDB Dataset.csv')

In [3]:

positive_samples = df[df['sentiment'] == 'positive'].sample(n=50, random_state=42)
negative_samples = df[df['sentiment'] == 'negative'].sample(n=50, random_state=42)
dataset = pd.concat([positive_samples, negative_samples])

dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)
dataset.to_csv('reviews_100.csv', index=False)


In [4]:
print(dataset['sentiment'].value_counts())

negative    50
positive    50
Name: sentiment, dtype: int64


Stopwords removal and tokenization


In [13]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/somyadahiaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/somyadahiaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

# dataset['joined_review'] = dataset['cleaned_review'].apply(lambda words: ' '.join(words))

# vectorizer = CountVectorizer(max_features=500, stop_words='english')
# X = vectorizer.fit_transform(dataset['joined_review'])  # Features
# y = dataset['sentiment']  # Labels






stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    text =re.sub(r'<[^>]+>', '', text)
    remove_punctuation=[char for char in text if char not in string.punctuation]
    remove_punctuation=''.join(remove_punctuation)
    text= [word for word in remove_punctuation.split() if word.lower() not in stop_words]
    return text

# Test
# print(remove_stopwords("This is an example <br> text<\br>."))

['example', 'text']


In [54]:


dataset.iloc[:,0].apply(remove_stopwords)


0     [viewed, first, two, nights, coming, IMDb, loo...
1     [said, several, times, least, watching, Eric, ...
2     [Without, doubt, WORSE, comicbook, movie, ever...
3     [Firstly, would, like, point, people, criticis...
4     [compact, crime, drama, good, amount, action, ...
                            ...                        
95    [wasnt, saw, Sidney, Pollack, picture, ever, c...
96    [Daniel, Auteuil, Queen, Margot, much, better,...
97    [got, movie, eBay, mainly, Im, gay, love, Til,...
98    [Preposterous, sequel, stretches, credibility,...
99    [Bugs, Bunny, accidentally, ends, South, Pole,...
Name: review, Length: 100, dtype: object

In [107]:
from sklearn.feature_extraction.text import CountVectorizer

transformer=CountVectorizer(analyzer=remove_stopwords, max_features=500).fit(dataset['review'])

vectorised=transformer.transform(dataset['review'])

In [108]:
X=vectorised
dataset['label'] = dataset['sentiment'].map({'positive': 1, 'negative': 0})
y=dataset['label']
print(X.shape)


(100, 500)


In [109]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [110]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model = MultinomialNB()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5000


In [111]:
def predict_review_sentiment(model, vectorizer, review):
    review_vector = vectorizer.transform([review])
    prediction = model.predict(review_vector)[0]
    return prediction

In [119]:
review_texts = ["An absolutely stunning movie with a powerful message!","Absolutely amazing work, good movie.", "Worst movie ever, not recommended", "good movie , thriller and intersting to watch with friends." ]

for review in review_texts:
    predicted=predict_review_sentiment(model, transformer, review)
    print(review)

    if predicted==1:
        print("Predicted Sentiment: positive")

    else:
        print("Predicted Sentiment: negative")

       





An absolutely stunning movie with a powerful message!
Predicted Sentiment: positive
Absolutely amazing work, good movie.
Predicted Sentiment: positive
Worst movie ever, not recommended
Predicted Sentiment: negative
good movie , thriller and intersting to watch with friends.
Predicted Sentiment: positive
